In [28]:
import os
import json
import numpy as np
from omegaconf import OmegaConf

In [29]:
exp_dir = "../../../logs/en/asr/evaluate"
folders = {
    "Baseline": "version_5",
    "Fine-tuned_9": "version_9",
    "AC_66": "version_24",
    "x_vector": "version_55",
}

In [30]:
# define which accent is standard, which are seen and which are unseen
classes = {
    "seen": ["us", "uk", "ca", "in", "de", "au"],
    "unseen": ["hk", "sg", "sc", "nz", "ie", "za", "ni", "ph"],
}

In [31]:
# load WERs of each file, for each accent
wers = dict()
for exp, folder in folders.items():
    wers[exp] = {k[5:-4]: v for k, v in json.load(open(f"{exp_dir}/{folder}/avg_wers.json")).items()}

In [32]:
# transform dictionary into a numpy array, where one axis represents the accents and another the experiments
experiments = list(wers.keys())
accents = list(wers[experiments[0]].keys())
n_words = np.array([wers[experiments[0]][acc]["n_words"] for acc in accents])
avg_wers = np.array([[wers[exp][acc]["avg_wer"] for exp in experiments] for acc in accents])

In [33]:
# compute means for each class, for each experiment
class_indices = {k: [accents.index(acc) for acc in v] for k, v in classes.items()}
class_means = {k: np.mean(avg_wers[indices], axis=0) for k, indices in class_indices.items()}

In [34]:
# print the avg. WERs and the means (overall and per class) as a markdown table
headers = ["Accent / Dialect"] + experiments
print((" | ").join(headers))
print(f"|{'---:|'*len(headers)}")

# print avg. WERs
for i in range(len(accents)):
    row = [accents[i]] + [f"{avg_wers[i,j]*100:.2f}" for j in range(len(experiments))]
    print((" | ").join(row))

# print overall means
row = ["mean"] + [f"{np.mean(avg_wers[:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))

# print class means
for key, value in class_means.items():
    row = [f"{key} mean"] + [f"{value[i]*100:.2f}" for i in range(len(experiments))]
    print((" | ").join(row))

# print worst avg. WER of each experiment
row = ["worst"] + [f"{v*100:.2f}" for v in np.max(avg_wers, axis=0)]
print((" | ").join(row))

# print the avg. of the three worst avg. WERs of each experiment
row = ["3-worst mean."] + [f"{np.mean(np.sort(avg_wers, axis=0)[-3:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))


Accent / Dialect | Baseline | Fine-tuned_9 | AC_66 | x_vector
|---:|---:|---:|---:|---:|
za | 8.54 | 6.54 | 6.54 | 6.47
sg | 12.11 | 10.41 | 10.51 | 10.45
in | 13.08 | 9.21 | 9.41 | 9.18
au | 10.21 | 7.47 | 7.53 | 7.44
hk | 12.68 | 9.76 | 9.75 | 9.71
ie | 8.96 | 7.51 | 7.41 | 7.40
uk | 7.50 | 5.78 | 5.87 | 5.77
ph | 10.44 | 8.90 | 8.79 | 8.79
de | 8.45 | 5.62 | 5.66 | 5.57
ni | 6.60 | 6.96 | 7.02 | 6.79
us | 6.88 | 5.43 | 5.46 | 5.45
ca | 6.30 | 5.36 | 5.35 | 5.32
nz | 6.82 | 5.40 | 5.28 | 5.31
sc | 38.48 | 33.50 | 32.55 | 33.59
mean | 11.22 | 9.13 | 9.08 | 9.09
seen mean | 8.74 | 6.48 | 6.55 | 6.46
unseen mean | 13.08 | 11.12 | 10.98 | 11.06
worst | 38.48 | 33.50 | 32.55 | 33.59
3-worst mean. | 21.41 | 17.89 | 17.61 | 17.92


Accent / Dialect | Baseline | Fine-tuned_6 | Fine-tuned_9 | AC_66 | x_vector_old | x_vector
|---:|---:|---:|---:|---:|---:|---:|
Southern African (South Africa, Zimbabwe, Namibia) | 8.54 | 8.05 | 6.54 | 6.54 | 6.49 | 6.47
Singaporean English | 12.11 | 12.06 | 10.41 | 10.51 | 10.41 | 10.45
India and South Asia (India, Pakistan, Sri Lanka) | 13.08 | 11.58 | 9.21 | 9.41 | 9.25 | 9.18
Australian English | 10.21 | 9.49 | 7.47 | 7.53 | 7.49 | 7.44
Hong Kong English | 12.68 | 11.45 | 9.76 | 9.75 | 9.70 | 9.71
Irish English | 8.96 | 8.78 | 7.51 | 7.41 | 7.43 | 7.40
England English | 7.50 | 7.18 | 5.78 | 5.87 | 5.79 | 5.77
Filipino | 10.44 | 10.38 | 8.90 | 8.79 | 8.81 | 8.79
German English,Non native speaker | 8.45 | 7.27 | 5.62 | 5.66 | 5.58 | 5.57
Northern Irish | 6.60 | 7.67 | 6.96 | 7.02 | 6.87 | 6.79
United States English | 6.88 | 6.67 | 5.43 | 5.46 | 5.42 | 5.45
Canadian English | 6.30 | 6.48 | 5.36 | 5.35 | 5.33 | 5.32
New Zealand English | 6.82 | 6.79 | 5.40 | 5.28 | 5.36 | 5.31
Scottish English | 38.48 | 34.03 | 33.50 | 32.55 | 33.65 | 33.59
mean | 11.22 | 10.56 | 9.13 | 9.08 | 9.11 | 9.09
seen mean | 8.74 | 8.11 | 6.48 | 6.55 | 6.48 | 6.46
unseen mean | 13.08 | 12.40 | 11.12 | 10.98 | 11.09 | 11.06
worst | 38.48 | 34.03 | 33.50 | 32.55 | 33.65 | 33.59
3-worst mean. | 21.41 | 19.22 | 17.89 | 17.61 | 17.92 | 17.92

In [35]:
# print the experiment folders of each experiment (both train and eval folders)
print("Experiment folders:\n")
for exp, folder in folders.items():
    eval_folder = os.path.join(exp_dir, folder)
    eval_config = OmegaConf.load(os.path.join(eval_folder, "config.yaml"))
    train_folder = f'../{eval_config.asr.ckpt.replace("/checkpoints/last.ckpt", "")}'
    print(f"- {exp}: training `{train_folder[3:]}`, evaluation `{eval_folder[3:]}`")

Experiment folders:



AttributeError: 'NoneType' object has no attribute 'replace'